##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Estimator에서 Keras API로 마이그레이션하기

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/guide/migrate/migrating_estimator"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a></td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/guide/migrate/migrating_estimator.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행하기</a> </td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ko/guide/migrate/migrating_estimator.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소스 보기</a></td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ko/guide/migrate/migrating_estimator.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드하기</a> </td>
</table>

이 가이드는 TensorFlow 1의 `tf.estimator.Estimator` API에서 TensorFlow 2의 `tf.keras` API로 마이그레이션하는 방법을 보여줍니다. 먼저 `tf.estimator.Estimator`를 사용하여 훈련 및 평가용 기본 모델을 설정하고 실행합니다. 그런 다음 `tf.keras` API를 사용하여 TensorFlow 2에서 동일한 단계를 수행합니다. 또한 `tf.keras.Model`을 하위 클래스화하고 `tf.GradientTape`를 사용하여 훈련 단계를 사용자 정의하는 방법을 배우게 됩니다.

- TensorFlow 1에서 고수준 `tf.estimator.Estimator` API를 사용하면 모델을 훈련 및 평가하고 추론을 수행하고 모델을 저장할 수 있습니다(서비스 제공용).
- TensorFlow 2에서 Keras API를 사용하여 앞서 언급한 [모델 구축하기](https://www.tensorflow.org/guide/keras/custom_layers_and_models), 그래디언트 적용하기, [훈련하기](https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit), 평가하기, 예측하기와 같은 작업을 수행합니다.

(모델/체크포인트 저장 워크플로를 TensorFlow 2로 마이그레이션하려면 [SavedModel](saved_model.ipynb) 및 [체크포인트](checkpoint_saved.ipynb) 마이그레이션 가이드를 확인하세요.)

## 설치하기

먼저 가져오기 및 간단한 데이터세트로 시작합니다.

In [ ]:
import tensorflow as tf
import tensorflow.compat.v1 as tf1

In [ ]:
features = [[1., 1.5], [2., 2.5], [3., 3.5]]
labels = [[0.3], [0.5], [0.7]]
eval_features = [[4., 4.5], [5., 5.5], [6., 6.5]]
eval_labels = [[0.8], [0.9], [1.]]

## TensorFlow 1: tf.estimator.Estimator로 훈련 및 평가하기

이 예제는 TensorFlow 1에서 `tf.estimator.Estimator`로 훈련과 평가를 수행하는 방법을 보여줍니다.

먼저 훈련 데이터용 입력 함수, 평가 데이터용 평가 입력 함수, 훈련 연산이 특성 및 레이블로 정의되는 방식을 `Estimator`에 알려주는 모델 함수 등 몇 가지 함수를 정의하는 것으로 시작합니다.

In [ ]:
def _input_fn():
return tf1.data.Dataset.from_tensor_slices((features, labels)).batch(1)

def _eval_input_fn():
return tf1.data.Dataset.from_tensor_slices(
(eval_features, eval_labels)).batch(1)

def _model_fn(features, labels, mode):
logits = tf1.layers.Dense(1)(features)
loss = tf1.losses.mean_squared_error(labels=labels, predictions=logits)
optimizer = tf1.train.AdagradOptimizer(0.05)
train_op = optimizer.minimize(loss, global_step=tf1.train.get_global_step())
return tf1.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)

`Estimator`를 인스턴스화하고 모델을 훈련합니다.

In [ ]:
estimator = tf1.estimator.Estimator(model_fn=_model_fn)
estimator.train(_input_fn)

평가 세트로 프로그램을 평가합니다.

In [ ]:
estimator.evaluate(_eval_input_fn)

## TensorFlow 2: 내장 Keras 메서드로 훈련 및 평가하기

이 예제는 TensorFlow 2에서 Keras `Model.fit`과 `Model.evaluate`로 훈련과 평가를 수행하는 방법을 보여줍니다(자세한 내용은 [내장 메서드를 사용하여 훈련 및 평가하기](https://www.tensorflow.org/guide/keras/train_and_evaluate) 가이드 참조).

- 먼저 `tf.data.Dataset` API로 데이터세트 파이프라인을 준비합니다.
- 하나의 선형(`tf.keras.layers.Dense`) 레이어로 간단한 Keras [순차형](https://www.tensorflow.org/guide/keras/sequential_model) 모델을 정의합니다.
- Adagrad 옵티마이저(`tf.keras.optimizers.Adagrad`)를 인스턴스화합니다
- `optimizer` 변수와 평균 제곱 오차(`"mse"`) 손실을 `Model.compile`에 전달하여 훈련용 모델을 구성합니다.

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((features, labels)).batch(1)
eval_dataset = tf.data.Dataset.from_tensor_slices(
      (eval_features, eval_labels)).batch(1)

model = tf.keras.models.Sequential([tf.keras.layers.Dense(1)])
optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.05)

model.compile(optimizer=optimizer, loss="mse")

이제 `Model.fit`을 호출하여 모델을 훈련할 준비가 된 것입니다.

In [ ]:
model.fit(dataset)

마지막으로 `Model.evaluate`를 사용하여 모델을 평가합니다.

In [ ]:
model.evaluate(eval_dataset, return_dict=True)

## TensorFlow 2: 사용자 정의 훈련 단계 및 내장 Keras 메서드로 훈련 및 평가하기

TensorFlow 2에서는 `tf.GradientTape`를 사용하여 고유한 사용자 정의 훈련 단계 함수를 작성하여 순방향 및 역방향 전달을 수행하고 `tf.keras.callbacks.Callback`과 `tf.distribute.Strategy`와 같은 내장 훈련 지원을 계속 활용할 수 있습니다(자세한 내용은 [Model.fit에서 발생하는 사용자 정의 설정](https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit) 및 [처음부터 사용자 정의 훈련 루프 작성하기](https://www.tensorflow.org/guide/keras/writing_a_training_loop_from_scratch) 참조).

이 예제에서는 `Model.train_step`을 재정의하는 `tf.keras.Sequential`을 하위 클래스화하여 사용자 정의 `tf.keras.Model`을 생성하는 것으로 시작합니다(자세한 내용은 [tf.keras.Model 하위 클래스화하기](https://www.tensorflow.org/guide/keras/custom_layers_and_models) 참조). 해당 클래스에서 각 데이터 배치에 대해 하나의 훈련 단계 동안 순방향 전달 및 역방향 전달을 수행하는 사용자 정의 `train_step` 함수를 정의합니다.


In [ ]:
class CustomModel(tf.keras.Sequential):
  """A custom sequential model that overrides `Model.train_step`."""

  def train_step(self, data):
    batch_data, labels = data

    with tf.GradientTape() as tape:
      predictions = self(batch_data, training=True)
      # Compute the loss value (the loss function is configured
      # in `Model.compile`).
      loss = self.compiled_loss(labels, predictions)

    # Compute the gradients of the parameters with respect to the loss.
    gradients = tape.gradient(loss, self.trainable_variables)
    # Perform gradient descent by updating the weights/parameters.
    self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
    # Update the metrics (includes the metric that tracks the loss).
    self.compiled_metrics.update_state(labels, predictions)
    # Return a dict mapping metric names to the current values.
    return {m.name: m.result() for m in self.metrics}

다음에서는 이전과 같은 작업을 수행합니다.

- `tf.data.Dataset`를 사용하여 데이터세트 파이프라인을 준비합니다.
- 하나의 `tf.keras.layers.Dense` 레이어로 간단한 모델을 정의합니다.
- Adagrad 인스턴스화(`tf.keras.optimizers.Adagrad`)를 수행합니다.
- 평균 제곱 오차(`"mse"`)를 손실 함수로 사용하는 한편 `Model.compile`을 사용하여 훈련용 모델을 구성합니다.

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((features, labels)).batch(1)
eval_dataset = tf.data.Dataset.from_tensor_slices(
      (eval_features, eval_labels)).batch(1)

model = CustomModel([tf.keras.layers.Dense(1)])
optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.05)

model.compile(optimizer=optimizer, loss="mse")

`Model.fit`을 호출하여 모델을 훈련합니다.

In [ ]:
model.fit(dataset)

마지막으로 `Model.evaluate`를 사용하여 프로그램을 평가합니다.

In [ ]:
model.evaluate(eval_dataset, return_dict=True)

## 다음 단계

유용할 수 있는 추가 Keras 리소스:

- 가이드: [내장 메서드를 사용하여 훈련 및 평가하기](https://www.tensorflow.org/guide/keras/train_and_evaluate)
- 가이드: [Model.fit의 동작 사용자 정의하기](https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit)
- 가이드: [훈련 루프 처음부터 작성하기](https://www.tensorflow.org/guide/keras/writing_a_training_loop_from_scratch)
- 가이드: [하위 클래스화를 통한 새로운 Keras 레이어 및 모델 만들기](https://www.tensorflow.org/guide/keras/custom_layers_and_models)

다음 가이드는 `tf.estimator` API에서 분산 전략 워크플로를 마이그레이션하는 데 도움이 될 수 있습니다.

- [TPUEstimator에서 TPUStrategy로 마이그레이션하기](tpu_estimator.ipynb)
- [단일 작업자 멀티 GPU 훈련 마이그레이션하기](mirrored_strategy.ipynb)
- [멀티 작업자 CPU/GPU 훈련 마이그레이션하기](multi_worker_cpu_gpu_training.ipynb)